# Exercises 12: Solution to Implementing a Complete Ontology

In [1]:
from owlready2 import *
onto = get_ontology("http://www.dummy.info/new.owl")

Define the entities

In [2]:
with onto:
    
    class Disease(Thing):
        pass

    class Treatment(Thing):
        pass

    class Symptom(Thing):
        pass

Define the relations

In [3]:
with onto:

    class has_symptom(ObjectProperty):
        domain = [Disease]
        range = [Symptom]

    class should_not_take(ObjectProperty):
        domain = [Disease, Symptom, Treatment]
        range = [Treatment]

    class should_not_be_taken_by(ObjectProperty):
        domain = [Treatment]
        range = [Disease, Symptom, Treatment]
        inverse_property = should_not_take
        
    class not_symptom_of(ObjectProperty):
        domain = [Symptom]
        range = [Disease]

    class treats(ObjectProperty):
        domain = [Treatment]
        range = [Symptom]


Add some data

In [4]:
acid = Disease(name='Acidulitis')
brac = Disease(name='Brachioma')
cali = Disease(name='Calicosis')
dive = Disease(name='Divertigo')
ecno = Disease(name='Ecnomia')

In [5]:
feve = Symptom(name='Fever')
glan = Symptom(name='Glandular swelling')
head = Symptom(name='Headache')
itch = Symptom(name='Itching')
jaun = Symptom(name='Jaundice')

In [6]:
kalo = Treatment(name='Kaloxin')
lora = Treatment(name='Lorapix')
mani = Treatment(name='Manigel')
nimb = Treatment(name='Nimbalex')
opti = Treatment(name='Optinox')

Define the relations

In [7]:
#* The symptoms of Acidulitis are Headache and/or Itching, without Glandular Swelling.
acid.has_symptom = [head,itch]
glan.not_symptom_of = [acid]
#* One of Brachioma's symptoms is Fever.
brac.has_symptom = [feve]
#* The symptoms of Divertigo are Glandular Swelling with either Itching and/or Jaundice.
dive.has_symptom = [glan, itch, jaun]
#* Patients with Calicosis should not take Optinox.
cali.should_not_take = [opti]
#* The symptoms of Ecnomia's are Fever with Glandular Swelling and/or Jaundice.
ecno.has_symptom = [feve, glan, jaun]
#* Itching is a symptom of either Divertigo or Ecnomia.
dive.has_symptom = [itch]
ecno.has_symptom = [itch]
itch.not_symptom_of = [acid, brac, cali]
#* Kaloxin should not be used if Fever is a symptom.
feve.should_not_take = [kalo]
#* Lorapix should not be used if Jaundice is a symptom.
jaun.should_not_take = [lora]
#* Manigel and Optinox should never be used together.
mani.should_not_take = [opti]
#* Headaches can be treated with either Lorapix or Nimbalex.
lora.treats = [head]
nimb.treats = [head]
#* Headache is not a symptom of Ecnomia.
head.not_symptom_of = [ecno]
#* Kaloxin and Nimbalex should never be used together.
kalo.should_not_take = [nimb]


Now for a query. What medicines treat each disease?

In [8]:
list(default_world.sparql(
    """
    PREFIX ONTO: <http://www.dummy.info/new.owl#>
    SELECT ?t ?d
    WHERE{
        ?d ONTO:has_symptom ?s
        ?t ONTO:treats ?s
    }
    """
))

[[new.Lorapix, new.Acidulitis], [new.Nimbalex, new.Acidulitis]]